# Spark UDFs

UDF (User Defined Function) são funções customizadas que podemos aplicar sobre colunas do Spark DataFrame.

## Vamos criar um DataFrame de exemplo

In [1]:
import pandas as pd

In [2]:
pandas_df = pd.DataFrame({
  'id': [1, 2, 3, 4],
  'age': [14, 15, 16, 16],
  'name': ['Barbara Maria', 'Barbara Beatriz', 'Cris', 'Danielle']})

df = spark.createDataFrame(pandas_df)

In [3]:
df.show()

+---+---+---------------+
| id|age|           name|
+---+---+---------------+
|  1| 14|  Barbara Maria|
|  2| 15|Barbara Beatriz|
|  3| 16|           Cris|
|  4| 16|       Danielle|
+---+---+---------------+



In [4]:
df.printSchema()

root
 |-- id: long (nullable = true)
 |-- age: long (nullable = true)
 |-- name: string (nullable = true)



## Exemplos

### Idade no ano que vem

Vamos criar uma `udf` que retorna a idade no ano seguinte (ou seja, a idade somada a um).

Primeiro, definimos uma função `soma_um`, que soma 1 a um número

In [5]:
def soma_um(num):
    return num + 1

Para criar a UDF, basta passar dois parâmetros para a função `udf`: a função e o tipo de retorno dela. Como idade é um campo inteiro, sabemos que o retorno dessa função será um número inteiro (`T.IntegerType()` - ou poderia também ser `T.LongType()`, que também é um tipo inteiro, mas para números maiores).

In [6]:
import pyspark.sql.functions as F
import pyspark.sql.types as T

In [7]:
soma_um_udf = F.udf(soma_um, T.IntegerType())

Aplicando `soma_um_udf` na coluna `age`, criamos uma nova coluna chamada `age_plus_one`:

In [8]:
df = df.withColumn('age_plus_one', soma_um_udf('age'))

In [9]:
df.show()

+---+---+---------------+------------+
| id|age|           name|age_plus_one|
+---+---+---------------+------------+
|  1| 14|  Barbara Maria|          15|
|  2| 15|Barbara Beatriz|          16|
|  3| 16|           Cris|          17|
|  4| 16|       Danielle|          17|
+---+---+---------------+------------+



**Observação**: Note que, neste caso, não precisaríamos (e **não deveríamos**) construir uma `udf` para fazer essa operação. Conseguimos fazer essa operação de adicionar 1 a um número da seguinte forma:

In [10]:
df.withColumn('age_plus_one', F.col('age') + 1).show()

+---+---+---------------+------------+
| id|age|           name|age_plus_one|
+---+---+---------------+------------+
|  1| 14|  Barbara Maria|          15|
|  2| 15|Barbara Beatriz|          16|
|  3| 16|           Cris|          17|
|  4| 16|       Danielle|          17|
+---+---+---------------+------------+



### Primeiro nome

Vamos criar uma coluna chamada `first_name` que conterá o primeiro nome, segundo a coluna `name` de cada um dos registros.

In [11]:
def get_first_name(name):
    return name.split(' ')[0]

Nesse caso, nossa função retorna uma `string`, assim vamos utilizar `T.StringType()` como segundo parâmetro da `F.udf`.

In [12]:
get_first_name_udf = F.udf(get_first_name, T.StringType())

In [13]:
df = df.withColumn('first_name', get_first_name_udf('name'))

In [14]:
df.show()

+---+---+---------------+------------+----------+
| id|age|           name|age_plus_one|first_name|
+---+---+---------------+------------+----------+
|  1| 14|  Barbara Maria|          15|   Barbara|
|  2| 15|Barbara Beatriz|          16|   Barbara|
|  3| 16|           Cris|          17|      Cris|
|  4| 16|       Danielle|          17|  Danielle|
+---+---+---------------+------------+----------+



**Observação**: Note que, neste caso, também, não precisaríamos (e **não deveríamos**) construir uma `udf` para fazer essa operação. Conseguimos fazer essa operação de capturar o primeiro nome da seguinte forma:

In [15]:
df.withColumn('first_name', F.split('name', ' ')[0]).show()

+---+---+---------------+------------+----------+
| id|age|           name|age_plus_one|first_name|
+---+---+---------------+------------+----------+
|  1| 14|  Barbara Maria|          15|   Barbara|
|  2| 15|Barbara Beatriz|          16|   Barbara|
|  3| 16|           Cris|          17|      Cris|
|  4| 16|       Danielle|          17|  Danielle|
+---+---+---------------+------------+----------+



## Exercício: crie e aplique uma udf que conta a quantidade de caracteres de cada nome

In [16]:
###

Assim como nos outros casos, existe uma função pronta do spark que já faz essa operação. Você consegue descobrir qual é e checar se ela realmente tem o mesmo comportamento da udf criada?

In [17]:
###